In [2]:
import scanpy as sc
#import scanpy.external as sce
from scipy.sparse import csr_matrix, vstack
from scipy.io import mmread, mmwrite
import scanorama
import anndata as ad
import numpy as np
from matplotlib.pyplot import rc_context
import pandas as pd
from sklearn.neighbors import NearestNeighbors

Load single-cell RNA-seq data from an H5AD file into an AnnData object.

The file 'data/original_raw.h5ad' contains the dataset, which is stored in `adata`.

In [4]:
# Assuming 'path_to_file.h5ad' is the path to your H5AD file

# Load the H5AD file into an AnnData object
adata = ad.read_h5ad('data/original_raw.h5ad')

Normalize the count data in the AnnData object, apply log transformation, and identify highly variable genes. 
 
The top 2000 genes are selected using the Seurat flavor, with batch correction based on the "origin" key.

In [8]:
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000, batch_key = "origin", inplace=True)

c:\Users\Elliott\AppData\Local\Programs\Python\Python39\lib\site-packages\anndata\_core\anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
c:\Users\Elliott\AppData\Local\Programs\Python\Python39\lib\site-packages\scanpy\preprocessing\_highly_variable_genes.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
c:\Users\Elliott\AppData\Local\Programs\Python\Python39\lib\site-packages\anndata\_core\anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
c:\Users\

Subset the AnnData object to retain only highly variable genes.  

The resulting object, `adata_hvg`, contains only the genes marked as highly variable in the `adata.var` metadata.

In [16]:
# Subset the AnnData object to contain only highly variable genes
adata_hvg = adata[:, adata.var['highly_variable']]

highly_variable_intersection
False    28312
True       371
Name: count, dtype: int64

Split the AnnData object into three subsets based on the "origin" column in the observations metadata.  
- `cao`: Contains data from the origin "Cao".  
- `dis`: Contains data from the origin "Disteche".  
- `ian`: Contains data from the origin "Ian".  
Each subset is a copy of the original data.

In [5]:
cao = adata[adata.obs['origin'] == "Cao"].copy()
dis = adata[adata.obs['origin'] == "Disteche"].copy()
ian = adata[adata.obs['origin'] == "Ian"].copy()

c:\Users\Elliott\AppData\Local\Programs\Python\Python39\lib\site-packages\anndata\_core\anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


Deletes adata to save memory

In [7]:
del(adata)

Split the highly variable gene subset (`adata_hvg`) into three datasets based on the "origin" column.  
- `cao_hvg`: Contains data from the origin "Cao".  
- `dis_hvg`: Contains data from the origin "Disteche".  
- `ian_hvg`: Contains data from the origin "Ian".  
Each subset is a copy of the highly variable gene data.

In [18]:
# Separate the three datasets
cao_hvg = adata_hvg[adata_hvg.obs['origin'] == "Cao"].copy()
dis_hvg = adata_hvg[adata_hvg.obs['origin'] == "Disteche"].copy()
ian_hvg = adata_hvg[adata_hvg.obs['origin'] == "Ian"].copy()

c:\Users\Elliott\AppData\Local\Programs\Python\Python39\lib\site-packages\anndata\_core\anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


Randomly sample 300,000 cells from the "Cerebellum" and 300,000 cells from the "Cerebrum" in the `cao_hvg` dataset.  

The resulting subset, `cao_hvg_600`, contains a total of 600,000 cells, evenly split between the two organs.

In [19]:
## Selects 300k cerebellum and 300k cerebrum cells from the Cao dataset

# Assuming you have an AnnData object called cao_hvg 

# Get indices for cells belonging to "Cerebellum" and "Cerebrum"
cerebellum_indices = np.where(cao_hvg.obs['Organ'] == 'Cerebellum')[0]
cerebrum_indices = np.where(cao_hvg.obs['Organ'] == 'Cerebrum')[0]

# Randomly sample 300k cells from each category
np.random.shuffle(cerebellum_indices)
np.random.shuffle(cerebrum_indices)
sample_size = 300000
sampled_indices = np.concatenate([cerebellum_indices[:sample_size], cerebrum_indices[:sample_size]])

# Subset the original AnnData object to contain only the sampled cells
cao_hvg_600 = cao_hvg[sampled_indices, :]

# Now sampled_adata contains 300k cells from both "Cerebellum" and "Cerebrum"

c:\Users\Elliott\AppData\Local\Programs\Python\Python39\lib\site-packages\anndata\_core\anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


Save the AnnData objects (`cao_hvg_600`, `dis_hvg`, `ian_hvg`) to H5AD files for future use.  
- `cao_hvg_600` is saved as `data/cao_hvg_600.h5ad`.  
- `dis_hvg` is saved as `data/dis_hvg.h5ad`.  
- `ian_hvg` is saved as `data/ian_hvg.h5ad`.

In [22]:
# Write the AnnData object to an H5AD file
cao_hvg_600.write_h5ad("data/cao_hvg_600.h5ad")
dis_hvg.write_h5ad("data/dis_hvg.h5ad")
ian_hvg.write_h5ad("data/ian_hvg.h5ad")